In [8]:
import finnhub
import secrets_stock_news
import datetime

In [9]:
finnhub_client = finnhub.Client(api_key=secrets_stock_news.API_KEY)

In [10]:
str(datetime.date.today())

'2023-06-17'

In [11]:
company_news = finnhub_client.company_news('AAPL', _from=str(datetime.date.today()), to=str(datetime.date.today()))

In [39]:
general_news = finnhub_client.general_news('general', min_id=0)

In [13]:
for news in general_news:
    print(news.keys())
    break

dict_keys(['category', 'datetime', 'headline', 'id', 'image', 'related', 'source', 'summary', 'url'])


In [14]:
len(general_news)

100

In [15]:
for n in general_news:
    print(n.keys())
    break

dict_keys(['category', 'datetime', 'headline', 'id', 'image', 'related', 'source', 'summary', 'url'])


In [34]:
id = []

for news in general_news:
    id.append(news['id'])

if(len(id)):
    print(max(id))

7272796


In [ ]:
while(True):
    general_news = finnhub_client.general_news('general', min_id=0)
    sleep(60)

## Spark

In [22]:
import os

from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [23]:
spark = SparkSession \
        .builder \
        .appName("Stock News") \
        .master("local[*]") \
        .config("spark.jars", os.getcwd() + "../jars2/commons-pool2-2.6.2.jar" + "," + os.getcwd() + "../jars2/kafka-clients-2.6.0.jar"
                + "," + os.getcwd() + "../jars2/spark-sql-kafka-0-10_2.12-3.0.1.jar" + "," + os.getcwd() + "../jars2/spark-token-provider-kafka-0-10_2.12-3.0.1.jar") \
        .appName("Stock News")\
        .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")
spark.conf.set("spark.sql.shuffle.partitions", 1)

In [24]:
KAFKA_TOPIC_NAME = "stock_news"
KAFKA_BOOTSTRAP_SERVER = "localhost:9092"

In [25]:
flower_df = spark \
        .readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVER) \
        .option("subscribe", KAFKA_TOPIC_NAME) \
        .option("startingOffsets", "earliest") \
        .load()

AnalysisException:  Failed to find data source: kafka. Please deploy the application as per the deployment section of "Structured Streaming + Kafka Integration Guide".        

In [27]:
general_news[0].items()

dict_items([('category', 'top news'), ('datetime', 1686864300), ('headline', 'FA Center: Which is worse: Lying about money or losing it?'), ('id', 7273201), ('image', 'https://images.mktw.net/im-700936/social'), ('related', ''), ('source', 'MarketWatch'), ('summary', 'Admitting what you’ve done — the decisions you’ve made and the losses you’ve incurred — can sting.'), ('url', 'https://www.marketwatch.com/story/which-is-worse-lying-about-money-or-losing-it-cfa06368')])

In [28]:
general_news[0].keys()

dict_keys(['category', 'datetime', 'headline', 'id', 'image', 'related', 'source', 'summary', 'url'])

## Postgres test

In [16]:
import psycopg2
# from secrets_stock_news import PSQL_HOST
import secrets_stock_news

In [32]:
conn = psycopg2.connect(
    host=secrets_stock_news.PSQL_HOST,
    database=secrets_stock_news.PSQL_DATABASE,
    user=secrets_stock_news.PSQL_USER,
    password=secrets_stock_news.PSQL_PASSWORD,
    port=secrets_stock_news.PSQL_PORT
)

In [37]:

insert_query = "INSERT INTO news_dev (category, datetime, headline, id, image, related, source, summary, url) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"

In [41]:
with conn.cursor() as cursor:
    try:
        # Execute multiple INSERT statements in a single transaction
        cursor.executemany(insert_query, [tuple(item.values()) for item in general_news])

        # Commit the transaction
        conn.commit()

        print("Data inserted successfully!")
    except (Exception, psycopg2.DatabaseError) as error:
        # Rollback the transaction in case of an error
        conn.rollback()
        print("Error occurred during data insertion:", error)

Data inserted successfully!


In [ ]:
conn.close()

In [36]:
general_news[0]

('top news',
 1687021260,
 ': Another Nvidia board member unloads stock, this time to the tune of $21 million',
 7273675,
 'https://images.mktw.net/im-683956/social',
 '',
 'MarketWatch',
 'Two Nvidia directors have sold valuable chunks of the high-flying stock this week, recent filings show.',
 'https://www.marketwatch.com/story/another-nvidia-board-member-unloads-stock-this-time-to-the-tune-of-21-million-62e9bfbd')